In [ ]:
#
# Started with voronoi_7.ipynb... but focused on collapsing the leftovers
#
import polars as pl
import numpy as np
import networkx as nx
import random
import rtsvg
rt = rtsvg.RACETrack()
circles     = [(25,25,20),(80,80,30),(20,100,10),(25,60,5),(60,30,5),(48,48,6),(40,90,6)]
circles = [(85,  53, 12), (77, 111,  5), (24,  58, 13), (19,  19, 13),
           (59,  21,  9), (93,  16, 10), (83,  79,  6), (16, 106,  5)]
svg_hdr     = '<svg x="0" y="0" width="768" height="768" viewbox="0 0 128 128"><rect x="0" y="0" width="128" height="128" fill="white" />'
svg_ftr     = '</svg>'
svg_circles = []
for i in range(len(circles)):
    svg_circles.append(f'<circle cx="{circles[i][0]}" cy="{circles[i][1]}" r="{circles[i][2]}" fill="none" stroke="{rt.co_mgr.getColor(i)}" stroke-width="0.4"/>')
    svg_circles.append(f'<circle cx="{circles[i][0]}" cy="{circles[i][1]}" r="0.5" fill="{rt.co_mgr.getColor(i)}"/>')

#rt.tile([svg_hdr + ''.join(svg_circles) + svg_ftr])

In [2]:
# Is a set of segments a triangle?
def isTriangle(_segments_, merge_threshold=0.5):
    if len(_segments_) < 3: return False
    xys = set()
    for _segment_ in _segments_: xys.add(_segment_[0]), xys.add(_segment_[1])
    xys_ls = list(xys)
    _centers_, _members_ = rt.kMeans2D(xys_ls, 3)
    # Are the clusters compact enough?
    for cluster_i in _centers_:
        _center_ = _centers_[cluster_i]
        for xy in _members_[cluster_i]:
            if rt.segmentLength((_center_, xy)) > merge_threshold: 
                return False
    # Do the edges form a triangle?
    # ... make a map from the points seen to their cluster numbers
    xy_to_cluster = {}
    for cluster_i in _members_: 
        for xy in _members_[cluster_i]: 
            xy_to_cluster[xy] = cluster_i
    # ... convert the edges to their cluster numbers
    _edges_seen_ = set()
    for _segment_ in _segments_:
        cluster_0 = xy_to_cluster[_segment_[0]]
        cluster_1 = xy_to_cluster[_segment_[1]]
        if cluster_0 == cluster_1: return False
        if cluster_0 > cluster_1: cluster_0, cluster_1 = cluster_1, cluster_0
        _edges_seen_.add((cluster_0, cluster_1))
    # Must be three edges exactly and they must be the edges of a triangle
    if len(_edges_seen_) != 3: return False
    if (0,1) not in _edges_seen_ or \
       (1,2) not in _edges_seen_ or \
       (0,2) not in _edges_seen_: return False
    return True

# return bisector of points p0 and p1 -- bisector is ((x,y),(u,v)) where u,v is the vector of the bisector
def _isedgar_bisector(p0, p1):
    x, y     = (p0[0] + p1[0])/2.0, (p0[1] + p1[1])/2.0
    uv       = rt.unitVector((p0, p1))
    pdx, pdy = uv[1], -uv[0]
    return ((x,y), (pdx,pdy))
# For the circle version
def _isedgar_bisectorForCircles(c0, c1):
    uv              = rt.unitVector((c0, c1))
    x0, y0          = c0[0] + uv[0] * c0[2], c0[1] + uv[1] * c0[2]
    x1, y1          = c1[0] - uv[0] * c1[2], c1[1] - uv[1] * c1[2]
    x,  y           = (x0+x1)/2.0, (y0+y1)/2.0
    pdx, pdy        = uv[1], -uv[0]
    return ((x,y), (pdx,pdy))
# returns vertices that intersect the polygon
def _isedgar_intersects(bisects, poly, merge_threshold):
    inters, already_added = [], set()
    for i in range(0, len(poly)):
        p0, p1 = poly[i], poly[(i+1)%len(poly)]
        xy = rt.lineSegmentIntersectionPoint((bisects[0], (bisects[0][0] + bisects[1][0], bisects[0][1] + bisects[1][1])),(p0, p1))
        if xy is not None:
            xy = (round(xy[0], 2), round(xy[1], 2))
            too_close = False
            for pt in already_added:
                if rt.segmentLength((pt, xy)) < merge_threshold:
                    too_close = True
                    break
            if too_close == False:
                inters.append((xy, i, (i+1)%len(poly)))
                already_added.add(xy)
    return inters
# conctenate a point, a list, and a point into a new list
def _isedgar_createCell(my_cell, p0, p1, i0, i1, sign):
    l = [p0]
    i = i0
    while i != i1:
        l.append(my_cell[i])
        i += sign
        if i < 0:             i += len(my_cell)
        if i >= len(my_cell): i -= len(my_cell)
    l.append(my_cell[i1])
    l.append(p1)
    return l
# contain true if pt is in poly
def _isedgar_contains(poly, pt):
    inter_count = 0
    for i in range(len(poly)):
        p0, p1 = poly[i], poly[(i+1)%len(poly)]
        _tuple_ = rt.segmentsIntersect((pt,(pt[0]+1e9,pt[1])),(p0,p1)) # use a ray from the pt to test
        if _tuple_[0] and (_tuple_[1], _tuple_[2]) != p1: inter_count += 1
    if inter_count%2 == 1: return True
    return False
# remove any near duplicate points from the polygon
def _isedgar_removeDuplicatePoints(_poly_, merge_threshold):
    _new_ = []
    _new_.append(_poly_[0])
    for i in range(1,len(_poly_)):
        if rt.segmentLength((_poly_[i],_new_[-1])) > merge_threshold: _new_.append(_poly_[i])
    return _new_
# normalize the segment
def _isedgar_normalizeSegment(segment):
    _xy0_, _xy1_ = segment
    if _xy0_[0] <  _xy1_[0] or (_xy0_[0] == _xy1_[0] and _xy0_[1] < _xy1_[1]): return (_xy0_, _xy1_)
    else:                                                                      return (_xy1_, _xy0_)
# clumsy ... but this should work / if not very elegantly...
def _isedgar_accountForChanges(_old_, _new_, _p_, s2p):
    segs_in_old, segs_in_new, xy_to_seg_in_old, xy_to_seg_in_new = set(), set(), {}, {}
    for i in range(len(_old_)):
        _segment_ = _isedgar_normalizeSegment((_old_[i], _old_[(i+1)%len(_old_)]))
        segs_in_old.add(_segment_)
        if _segment_[0] not in xy_to_seg_in_old: xy_to_seg_in_old[_segment_[0]] = set()
        if _segment_[1] not in xy_to_seg_in_old: xy_to_seg_in_old[_segment_[1]] = set()
        xy_to_seg_in_old[_segment_[0]].add(_segment_)
        xy_to_seg_in_old[_segment_[1]].add(_segment_)
    for i in range(len(_new_)):
        _segment_ = _isedgar_normalizeSegment((_new_[i], _new_[(i+1)%len(_new_)]))
        segs_in_new.add(_segment_)
        if _segment_[0] not in xy_to_seg_in_new: xy_to_seg_in_new[_segment_[0]] = set()
        if _segment_[1] not in xy_to_seg_in_new: xy_to_seg_in_new[_segment_[1]] = set()
        xy_to_seg_in_new[_segment_[0]].add(_segment_)
        xy_to_seg_in_new[_segment_[1]].add(_segment_)
    segs_only_in_old = segs_in_old - segs_in_new
    segs_only_in_new = segs_in_new - segs_in_old
    # new should be three -- two should be shortened (and share a single xy with the old set), one should be new (both xy's unique)
    if len(segs_only_in_new) != 3: raise ValueError('segs_only_in_new != 3')
    seg0, seg1, seg2 = list(segs_only_in_new)
    if   seg0[0] not in xy_to_seg_in_old and seg0[1] not in xy_to_seg_in_old: # seg0 is new
        seg_new, seg_single0, seg_single1 = seg0, seg1, seg2
    elif seg1[0] not in xy_to_seg_in_old and seg1[1] not in xy_to_seg_in_old: # seg1 is new
        seg_new, seg_single0, seg_single1 = seg1, seg0, seg2
    elif seg2[0] not in xy_to_seg_in_old and seg2[1] not in xy_to_seg_in_old: # seg2 is new
        seg_new, seg_single0, seg_single1 = seg2, seg0, seg1
    else: raise ValueError('no match [new segment]')
    # transfer points to both seg_single0 and seg_single1
    # ... find the two possible transfers for seg_single0
    if   seg_single0[0] in xy_to_seg_in_old:
        if seg_single0[1] in xy_to_seg_in_old: raise ValueError('seg_single0[1] in xy_to_seg_in_old (and should not be)')
        single0_old_seg0, single0_old_seg1 = list(xy_to_seg_in_old[seg_single0[0]])
    elif seg_single0[1] in xy_to_seg_in_old:
        if seg_single0[0] in xy_to_seg_in_old: raise ValueError('seg_single0[0] in xy_to_seg_in_old (and should not be)')
        single0_old_seg0, single0_old_seg1 = list(xy_to_seg_in_old[seg_single0[1]])
    else: raise ValueError('no match [seg_single0]')

    # ... find the two possible transfers for seg_single1
    if   seg_single1[0] in xy_to_seg_in_old:
        if seg_single1[1] in xy_to_seg_in_old: raise ValueError('seg_single1[1] in xy_to_seg_in_old (and should not be)')
        single1_old_seg0, single1_old_seg1 = list(xy_to_seg_in_old[seg_single1[0]])
    elif seg_single1[1] in xy_to_seg_in_old:
        if seg_single1[0] in xy_to_seg_in_old: raise ValueError('seg_single1[0] in xy_to_seg_in_old (and should not be)')
        single1_old_seg0, single1_old_seg1 = list(xy_to_seg_in_old[seg_single1[1]])
    else: raise ValueError('no match [seg_single1]')

    if (single0_old_seg0 == single1_old_seg0 and single0_old_seg1 == single1_old_seg1) or \
        (single0_old_seg0 == single1_old_seg1 and single0_old_seg1 == single1_old_seg0):
        # special case where the two segments are the same -- happens when the new cell is a triangle
        s2p[seg_single0] = s2p[single0_old_seg0]
        s2p[seg_single1] = s2p[single0_old_seg1]
    else:
        if   single0_old_seg0 in segs_in_new: s2p[seg_single0] = s2p[single0_old_seg1]
        elif single0_old_seg1 in segs_in_new: s2p[seg_single0] = s2p[single0_old_seg0]
        else: raise ValueError('at least one single0_old should be in segs_in_new')
        if   single1_old_seg0 in segs_in_new: s2p[seg_single1] = s2p[single1_old_seg1]
        elif single1_old_seg1 in segs_in_new: s2p[seg_single1] = s2p[single1_old_seg0]
        else: raise ValueError('at least one single1_old should be in segs_in_new')
# create a merge lookup table for xy's that are too close
def _isedgar_createMergerLookup(cells, merge_threshold):
    xys = set()
    for cell in cells:
        for xy in cell: xys.add(xy)
    xy_becomes = {}
    for xy0 in xys:
        for xy1 in xys:
            if xy0 == xy1 or xy1 in xy_becomes: continue
            if rt.segmentLength((xy0, xy1)) < merge_threshold: xy_becomes[xy1] = xy0
    return xy_becomes


In [3]:
#
# Algorithm source & description from:
# "A simple algorithm for 2D Voronoi diagrams"
# https://www.youtube.com/watch?v=I6Fen2Ac-1U
# https://gist.github.com/isedgar/d445248c9ff6c61cef44fc275cb2398f
#
def isedgarVoronoi(self, S, Box=None, pad=10, use_circle_radius=False, merge_threshold=0.5):
    if Box is None:
        x_l, x_r, y_t, y_b = S[0][0], S[0][0], S[0][1], S[0][1]
        for pt in S: x_l, x_r, y_t, y_b = min(x_l, pt[0]), max(x_r, pt[0]), max(y_t, pt[1]), min(y_b, pt[1])
        Box = [(x_l-pad, y_t+pad), (x_r+pad, y_t+pad), (x_r+pad, y_b-pad), (x_l-pad, y_b-pad)]
    cells, all_segments_to_points = [], {}
    for i in range(len(S)):
        p                  = S[i]
        cell               = Box
        segments_to_points = {}   # do this on a local basis and then group w/ others at the end
        for j in range(len(cell)):
            segment = _isedgar_normalizeSegment((cell[j], cell[(j+1)%len(cell)]))
            if segment not in segments_to_points: segments_to_points[segment] = set()
            segments_to_points[segment].add(p[:2])
        for q in S:
            if p == q: continue
            B            = _isedgar_bisector(p,q) if use_circle_radius == False else _isedgar_bisectorForCircles(p,q)
            B_intersects = _isedgar_intersects(B, cell, merge_threshold)
            if len(B_intersects) == 2:
                t1, t2  = B_intersects[0][0], B_intersects[1][0] # these are _xy_ tuples
                segment = _isedgar_normalizeSegment((t1, t2))
                if segment not in segments_to_points: segments_to_points[segment] = set()
                segments_to_points[segment].add(p[:2]), segments_to_points[segment].add(q[:2]) 
                xi, xj  = B_intersects[0][2], B_intersects[1][1]
                newCell = _isedgar_createCell(cell, t1, t2, xi, xj, 1)
                if _isedgar_contains(newCell, p) == False:
                    xi, xj  = B_intersects[0][1], B_intersects[1][2]
                    newCell = _isedgar_createCell(cell, t1, t2, xi, xj, -1)
                _isedgar_accountForChanges(cell, newCell, p[:2], segments_to_points)
                cell = newCell
        cells.append(cell)
        # transfer segments from local to all
        for j in range(len(cell)):
            segment = _isedgar_normalizeSegment((cell[j], cell[(j+1)%len(cell)]))
            if segment not in all_segments_to_points: all_segments_to_points[segment] = set()
            all_segments_to_points[segment].update(segments_to_points[segment])
        # remove any near duplicate points
        xy_becomes = _isedgar_createMergerLookup(cells, merge_threshold)
        new_cells = []
        for cell in cells:
            new_cell = []
            for i in range(len(cell)):
                if cell[i] in xy_becomes: new_cell.append(xy_becomes[cell[i]])
                else:                     new_cell.append(cell[i])
            new_cells.append(new_cell)
        cells = new_cells
        new_all_segments_to_points = {}
        for _segment_ in all_segments_to_points:
            if _segment_[0] in xy_becomes: _xy0_ = xy_becomes[_segment_[0]]
            else:                          _xy0_ = _segment_[0]
            if _segment_[1] in xy_becomes: _xy1_ = xy_becomes[_segment_[1]]
            else:                          _xy1_ = _segment_[1]
            new_all_segments_to_points[(_xy0_, _xy1_)] = all_segments_to_points[_segment_]
        all_segments_to_points = new_all_segments_to_points

    return cells, all_segments_to_points

In [ ]:
polys, s2p = isedgarVoronoi(rt, circles, [(0,0),(0,128),(128,128),(128,0)], merge_threshold=0.5, use_circle_radius=True)
svg_polys, xys_seen = [], set()
for i in range(len(polys)):
    poly = polys[i]
    d = [f'M {poly[0][0]} {poly[0][1]} ']
    xys_seen.add(poly[0])
    for j in range(1,len(poly)): 
        d.append(f'L {poly[j][0]} {poly[j][1]} ')
        xys_seen.add(poly[j])
    d.append('Z')
    svg_polys.append(f'<path d="{"".join(d)}" fill="none" stroke="{rt.co_mgr.getColor(i)}" stroke-width="0.5"/>')
for _xy_ in xys_seen:
    svg_polys.append(f'<circle cx="{_xy_[0]}" cy="{_xy_[1]}" r="{1.0 + random.random()}" fill="none" stroke="#000000" stroke-width="0.1"/>')
print(f'{len(xys_seen)=}') # 30 points seen (base version w/out any mergers)
                           # 27 if intersection points are rounded to 1 decimal place (27 also for 2 decimal place rounding)
min_segment_length = 1e9
for i in range(len(polys)):
    _poly_ = polys[i]
    for j in range(len(_poly_)):
        _xy0_, _xy1_ = _poly_[j], _poly_[(j+1)%len(_poly_)]
        min_segment_length = min(min_segment_length, rt.segmentLength((_xy0_, _xy1_)))
print(f'{min_segment_length=}') # 7.53... for no merger version
rt.tile([svg_hdr + ''.join(svg_circles) + ''.join(svg_polys) + svg_ftr])

In [ ]:
# collect all of the pieces that were not accounted for
pieces = set()
for i in range(len(circles)): # for all circles...
    _xy_, _poly_ = circles[i][:2], polys[i]
    for _segment_ in s2p: # for all of the recorded segments used
        if _xy_ in s2p[_segment_]: # if that segment related to this circles center...
            for j in range(len(_poly_)):
                _edge_ = (_poly_[j], _poly_[(j+1)%len(_poly_)])
                if rt.segmentsOverlap(_segment_, _edge_): break
                _edge_ = None
            if _edge_ is None: pieces.add(_isedgar_normalizeSegment(_segment_))
            else:
                diffs = rt.segmentDiffPieces(_segment_, _edge_)
                for _piece_ in diffs: pieces.add(_piece_)

def segmentsShareAPoint(_seg0_, _seg1_):
    # Try for an exact match w/ rounded numbers
    a, b = (round(_seg0_[0][0], 2), round(_seg0_[0][1], 2)), (round(_seg0_[1][0], 2), round(_seg0_[1][1], 2))
    c, d = (round(_seg1_[0][0], 2), round(_seg1_[0][1], 2)), (round(_seg1_[1][0], 2), round(_seg1_[1][1], 2))
    if a == c or a == d or b == c or b == d: return True
    # Otherwise, do a more expensive check
    l0, l1, l2, l3 = rt.segmentLength((a,c)), rt.segmentLength((a,d)), rt.segmentLength((b,c)), rt.segmentLength((b,d))
    if l0 < 0.2 or l1 < 0.2 or l2 < 0.2 or l3 < 0.2: return True
    # Otherwise, they don't share a point
    return False

# group them together into clusters / first iteration will be expensive
cluster_to_pieces = {}
pieces = list(pieces)
for i in range(len(pieces)):
    _piece_ = pieces[i]
    cluster_to_pieces[i] = set([_piece_])
for i in range(len(pieces)): # at most, we have to do n iterations
    clusters_merged, to_merge = False, None
    for c0 in cluster_to_pieces:
        for _seg0_ in cluster_to_pieces[c0]:
            for c1 in cluster_to_pieces:
                if c0 == c1: continue
                for _seg1_ in cluster_to_pieces[c1]:
                    if segmentsShareAPoint(_seg0_, _seg1_):
                        to_merge = (c0, c1)
                        clusters_merged = True
                        break
            if clusters_merged: break
        if clusters_merged: break
    if to_merge is not None:
        cluster_to_pieces[to_merge[0]].update(cluster_to_pieces[to_merge[1]])
        del cluster_to_pieces[to_merge[1]]
    if not clusters_merged: break

pieces_svg = []
for _piece_ in pieces:
    pieces_svg.append(f'<line x1="{_piece_[0][0]}" y1="{_piece_[0][1]}" x2="{_piece_[1][0]}" y2="{_piece_[1][1]}" stroke="#000000" stroke-width="0.2"/>')

cluster_pieces_svg = []
for c in cluster_to_pieces:
    _color_ = rt.co_mgr.getColor(c)
    for _piece_ in cluster_to_pieces[c]:
        cluster_pieces_svg.append(f'<line x1="{_piece_[0][0]}" y1="{_piece_[0][1]}" x2="{_piece_[1][0]}" y2="{_piece_[1][1]}" stroke="{_color_}" stroke-width="0.2"/>')
rt.tile([svg_hdr + ''.join(pieces_svg) + svg_ftr, svg_hdr + ''.join(cluster_pieces_svg) + svg_ftr],spacer=10)

In [ ]:
# Create a central point for each of the broken regions -- make a lookup
fixer_lu = {} # converts points that are "broken" into a new average point for that region
for c in cluster_to_pieces: 
    # print(c, isTriangle(cluster_to_pieces[c]))
    x_sum, y_sum, samples = 0.0, 0.0, 0
    for _piece_ in cluster_to_pieces[c]:
        for _xy_ in _piece_:
            x_sum, y_sum, samples = x_sum + _xy_[0], y_sum + _xy_[1], samples + 1
    x_avg, y_avg = x_sum / samples, y_sum / samples
    for _piece_ in cluster_to_pieces[c]:
        for _xy_ in _piece_:
            fixer_lu[_xy_] = (x_avg, y_avg)
# For each polygon, convert any points that are "broken"
new_polys = []
for _poly_ in polys:
    new_poly = []
    for _xy_ in _poly_:
        if _xy_ in fixer_lu: _xy_ = fixer_lu[_xy_]
        new_poly.append(_xy_)
    new_polys.append(new_poly)

# Render the example
xys_seen, svg_polys = set(), []
for i in range(len(new_polys)):
    poly = new_polys[i]
    d = [f'M {poly[0][0]} {poly[0][1]} ']
    xys_seen.add(poly[0])
    for j in range(1,len(poly)): 
        d.append(f'L {poly[j][0]} {poly[j][1]} ')
        xys_seen.add(poly[j])
    d.append('Z')
    svg_polys.append(f'<path d="{"".join(d)}" fill="none" stroke="{rt.co_mgr.getColor(i)}" stroke-width="0.5"/>')
for _xy_ in xys_seen: svg_polys.append(f'<circle cx="{_xy_[0]}" cy="{_xy_[1]}" r="{0.5 + 1.5 * random.random()}" fill="none" stroke="#000000" stroke-width="0.1"/>')

svg_actual = ['<g opacity="0.4">']
_inc_ = 0.5
x     = 0.0
while x < 128.0:
    y = 0.0
    while y < 128.0:
        closest_circle, closest_d = None, 1e9
        for i in range(len(circles)):
            _xy_ = circles[i][:2]
            d = ((x - _xy_[0])**2 + (y - _xy_[1])**2)**0.5 - circles[i][2]
            if d < closest_d: closest_d, closest_circle = d, i
        svg_actual.append(f'<rect x="{x}" y="{y}" width="{_inc_}" height="{_inc_}" stroke="none" fill="{rt.co_mgr.getColor(closest_circle)}"/>')
        y += _inc_
    x += _inc_
svg_actual.append('</g>')

#rt.tile([svg_hdr + ''.join(svg_actual) + ''.join(svg_circles) + ''.join(svg_polys) + svg_ftr])
rt.tile([svg_hdr + ''.join(svg_circles) + ''.join(svg_polys) + svg_ftr])

In [ ]:
#
# Put It All Together Now
#

#
# Algorithm source & description from:
# "A simple algorithm for 2D Voronoi diagrams"
# https://www.youtube.com/watch?v=I6Fen2Ac-1U
# https://gist.github.com/isedgar/d445248c9ff6c61cef44fc275cb2398f
#
def isedgarVoronoi_v2(self, S, Box=None, pad=10, use_circle_radius=False, merge_threshold=0.5):
    if Box is None:
        x_l, x_r, y_t, y_b = S[0][0], S[0][0], S[0][1], S[0][1]
        for pt in S: x_l, x_r, y_t, y_b = min(x_l, pt[0]), max(x_r, pt[0]), max(y_t, pt[1]), min(y_b, pt[1])
        Box = [(x_l-pad, y_t+pad), (x_r+pad, y_t+pad), (x_r+pad, y_b-pad), (x_l-pad, y_b-pad)]
    cells, all_segments_to_points = [], {}
    for i in range(len(S)):
        p                  = S[i]
        cell               = Box
        segments_to_points = {}   # do this on a local basis and then group w/ others at the end
        for j in range(len(cell)):
            segment = _isedgar_normalizeSegment((cell[j], cell[(j+1)%len(cell)]))
            if segment not in segments_to_points: segments_to_points[segment] = set()
            segments_to_points[segment].add(p[:2])
        for q in S:
            if p == q: continue
            B            = _isedgar_bisector(p,q) if use_circle_radius == False else _isedgar_bisectorForCircles(p,q)
            B_intersects = _isedgar_intersects(B, cell, merge_threshold)
            if len(B_intersects) == 2:
                t1, t2  = B_intersects[0][0], B_intersects[1][0] # these are _xy_ tuples
                segment = _isedgar_normalizeSegment((t1, t2))
                if segment not in segments_to_points: segments_to_points[segment] = set()
                segments_to_points[segment].add(p[:2]), segments_to_points[segment].add(q[:2]) 
                xi, xj  = B_intersects[0][2], B_intersects[1][1]
                newCell = _isedgar_createCell(cell, t1, t2, xi, xj, 1)
                if _isedgar_contains(newCell, p) == False:
                    xi, xj  = B_intersects[0][1], B_intersects[1][2]
                    newCell = _isedgar_createCell(cell, t1, t2, xi, xj, -1)
                _isedgar_accountForChanges(cell, newCell, p[:2], segments_to_points)
                cell = newCell
        cells.append(cell)
        # transfer segments from local to all
        for j in range(len(cell)):
            segment = _isedgar_normalizeSegment((cell[j], cell[(j+1)%len(cell)]))
            if segment not in all_segments_to_points: all_segments_to_points[segment] = set()
            all_segments_to_points[segment].update(segments_to_points[segment])
        # remove any near duplicate points
        xy_becomes = _isedgar_createMergerLookup(cells, merge_threshold)
        new_cells = []
        for cell in cells:
            new_cell = []
            for i in range(len(cell)):
                if cell[i] in xy_becomes: new_cell.append(xy_becomes[cell[i]])
                else:                     new_cell.append(cell[i])
            new_cells.append(new_cell)
        cells = new_cells
        new_all_segments_to_points = {}
        for _segment_ in all_segments_to_points:
            if _segment_[0] in xy_becomes: _xy0_ = xy_becomes[_segment_[0]]
            else:                          _xy0_ = _segment_[0]
            if _segment_[1] in xy_becomes: _xy1_ = xy_becomes[_segment_[1]]
            else:                          _xy1_ = _segment_[1]
            new_all_segments_to_points[(_xy0_, _xy1_)] = all_segments_to_points[_segment_]
        all_segments_to_points = new_all_segments_to_points

    if use_circle_radius:
        # collect all of the pieces that were not accounted for
        pieces = set()
        for i in range(len(S)): # for all circles...
            _xy_, _poly_ = S[i][:2], cells[i]
            for _segment_ in all_segments_to_points: # for all of the recorded segments used
                if _xy_ in all_segments_to_points[_segment_]: # if that segment related to this circles center...
                    for j in range(len(_poly_)):
                        _edge_ = (_poly_[j], _poly_[(j+1)%len(_poly_)])
                        if self.segmentsOverlap(_segment_, _edge_): break
                        _edge_ = None
                    if _edge_ is None: pieces.add(_isedgar_normalizeSegment(_segment_))
                    else:
                        diffs = self.segmentDiffPieces(_segment_, _edge_)
                        for _piece_ in diffs: pieces.add(_piece_)

        def segmentsShareAPoint(_seg0_, _seg1_):
            # Try for an exact match w/ rounded numbers
            a, b = (round(_seg0_[0][0], 2), round(_seg0_[0][1], 2)), (round(_seg0_[1][0], 2), round(_seg0_[1][1], 2))
            c, d = (round(_seg1_[0][0], 2), round(_seg1_[0][1], 2)), (round(_seg1_[1][0], 2), round(_seg1_[1][1], 2))
            if a == c or a == d or b == c or b == d: return True
            # Otherwise, do a more expensive check
            l0, l1, l2, l3 = self.segmentLength((a,c)), self.segmentLength((a,d)), self.segmentLength((b,c)), self.segmentLength((b,d))
            if l0 < 0.2 or l1 < 0.2 or l2 < 0.2 or l3 < 0.2: return True
            # Otherwise, they don't share a point
            return False

        # group them together into clusters / first iteration will be expensive
        cluster_to_pieces = {}
        pieces = list(pieces)
        for i in range(len(pieces)):
            _piece_ = pieces[i]
            cluster_to_pieces[i] = set([_piece_])
        for i in range(len(pieces)): # at most, we have to do n iterations
            clusters_merged, to_merge = False, None
            for c0 in cluster_to_pieces:
                for _seg0_ in cluster_to_pieces[c0]:
                    for c1 in cluster_to_pieces:
                        if c0 == c1: continue
                        for _seg1_ in cluster_to_pieces[c1]:
                            if segmentsShareAPoint(_seg0_, _seg1_):
                                to_merge = (c0, c1)
                                clusters_merged = True
                                break
                    if clusters_merged: break
                if clusters_merged: break
            if to_merge is not None:
                cluster_to_pieces[to_merge[0]].update(cluster_to_pieces[to_merge[1]])
                del cluster_to_pieces[to_merge[1]]
            if not clusters_merged: break

        # Create a central point for each of the broken regions -- make a lookup
        fixer_lu = {} # converts points that are "broken" into a new average point for that region
        for c in cluster_to_pieces: 
            # print(c, isTriangle(cluster_to_pieces[c]))
            x_sum, y_sum, samples = 0.0, 0.0, 0
            for _piece_ in cluster_to_pieces[c]:
                for _xy_ in _piece_:
                    x_sum, y_sum, samples = x_sum + _xy_[0], y_sum + _xy_[1], samples + 1
            x_avg, y_avg = x_sum / samples, y_sum / samples
            for _piece_ in cluster_to_pieces[c]:
                for _xy_ in _piece_:
                    fixer_lu[_xy_] = (x_avg, y_avg)
        # For each polygon, convert any points that are "broken"
        new_cells = []
        for _poly_ in cells:
            new_poly = []
            for _xy_ in _poly_:
                if _xy_ in fixer_lu: _xy_ = fixer_lu[_xy_]
                new_poly.append(_xy_)
            new_cells.append(new_poly)
        cells = new_cells

    return cells

polys = isedgarVoronoi_v2(rt, circles, [(0,0),(0,128),(128,128),(128,0)], merge_threshold=0.5, use_circle_radius=True)

svg_fixed, xys_seen = [], set()
for _poly_ in polys:
    d = [f'M {_poly_[0][0]} {_poly_[0][1]} ']
    xys_seen.add(_poly_[0])
    for j in range(1,len(_poly_)): 
        d.append(f'L {_poly_[j][0]} {_poly_[j][1]} ')
        xys_seen.add(_poly_[j])
    d.append('Z')
    svg_fixed.append(f'<path d="{"".join(d)}" fill="none" stroke="#000000" stroke-width="0.1"/>')
for _xy_ in xys_seen: svg_fixed.append(f'<circle cx="{_xy_[0]}" cy="{_xy_[1]}" r="{0.5 + 1.5 * random.random()}" fill="none" stroke="#000000" stroke-width="0.1"/>')

rt.tile([svg_hdr + ''.join(svg_fixed) + ''.join(svg_circles) + svg_ftr])

In [ ]:
from math import sqrt
def randomCircles(n_circles=8, w=128, h=128, r_min=5, r_max=15):
    circles = []
    for i in range(n_circles):
        circle_overlaps = True
        while circle_overlaps:
            r = random.randint(r_min, r_max)
            _possible_ = (random.randint(r+5, w-r-5), random.randint(r+5, h-r-5), r)
            circle_overlaps = False
            for _circle_ in circles:
                dx, dy = _circle_[0] - _possible_[0], _circle_[1] - _possible_[1]
                d      = sqrt(dx*dx+dy*dy)
                if d < (r + _circle_[2] + 5): # at least 10 pixels apart...
                    circle_overlaps = True
                    break
        circles.append(_possible_)
    return circles

_circles_ = randomCircles()
_polys_   = isedgarVoronoi_v2(rt, _circles_, [(0,0),(0,128),(128,128),(128,0)], merge_threshold=0.5, use_circle_radius=True)
_svg_, _xys_ = [], set()
for _poly_ in _polys_:
    d = [f'M {_poly_[0][0]} {_poly_[0][1]} ']
    _xys_.add(_poly_[0])
    for j in range(1,len(_poly_)): 
        d.append(f'L {_poly_[j][0]} {_poly_[j][1]} ')
        _xys_.add(_poly_[j])
    d.append('Z')
    _svg_.append(f'<path d="{"".join(d)}" fill="none" stroke="#000000" stroke-width="0.1"/>')
for _xy_ in _xys_: _svg_.append(f'<circle cx="{_xy_[0]}" cy="{_xy_[1]}" r="{0.5 + 1.5 * random.random()}" fill="none" stroke="#000000" stroke-width="0.1"/>')
for _circle_ in _circles_: _svg_.append(f'<circle cx="{_circle_[0]}" cy="{_circle_[1]}" r="{_circle_[2]}" fill="none" stroke="#000000" stroke-width="0.1"/>')

_polys_, _s2p_ = isedgarVoronoi(rt, _circles_, [(0,0),(0,128),(128,128),(128,0)], merge_threshold=0.5, use_circle_radius=True)
_svg2_, _xys_ = [], set()
for _poly_ in _polys_:
    d = [f'M {_poly_[0][0]} {_poly_[0][1]} ']
    _xys_.add(_poly_[0])
    for j in range(1,len(_poly_)): 
        d.append(f'L {_poly_[j][0]} {_poly_[j][1]} ')
        _xys_.add(_poly_[j])
    d.append('Z')
    _svg2_.append(f'<path d="{"".join(d)}" fill="none" stroke="#000000" stroke-width="0.1"/>')
for _xy_ in _xys_: _svg2_.append(f'<circle cx="{_xy_[0]}" cy="{_xy_[1]}" r="{0.5 + 1.5 * random.random()}" fill="none" stroke="#000000" stroke-width="0.1"/>')
for _circle_ in _circles_: _svg2_.append(f'<circle cx="{_circle_[0]}" cy="{_circle_[1]}" r="{_circle_[2]}" fill="none" stroke="#000000" stroke-width="0.1"/>')

rt.tile([svg_hdr + ''.join(_svg_) + svg_ftr, svg_hdr + ''.join(_svg2_) + svg_ftr], spacer=10)

In [ ]:
_circles_